In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import itertools
import random
from IPython.display import clear_output
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

## Ficheros y rutas de entrada/salida 

In [3]:
dir_in = '../../datos/datos_desarrollo'
file1_in = 'consumo_final.csv'
dir_out = dir_in

## Carga de datos 

In [4]:
df_consumo = pd.read_csv(os.path.join(dir_in, file1_in), sep = ';')
df_consumo.columns = [columna.lower() for columna in df_consumo.columns]
df_consumo.rename(columns = {'fecha_inicio': 'mes_inicio_temp', 'fecha_fin': 'mes_fin_temp'}, inplace = True)
df_consumo.date = df_consumo.date.astype('datetime64[ns]')

In [5]:
precio_model = df_consumo[['ccaa', 'producto', 'volumen_miles_de_kg', 'valor_miles_de_€', 'precio_medio_kg', 'date']]
cmpl_model_dict = {product: {comunidad: precio_model[(precio_model.producto.eq(product))&
                                                      (precio_model.ccaa.eq(comunidad))].drop(['producto', 'ccaa'], axis = 1)
                              for comunidad in precio_model.ccaa.unique()} for product in precio_model.producto.unique()}

In [6]:
def best_sarima(variable, p = 2, d = 1, q = 0, P = 0, D = 0, Q = 3, m = 12):
    try:
        len(predicciones)
    except NameError:
        predicciones = pd.DataFrame(columns = ['ccaa', 'producto'])
    vuelta_general = 1
    for k, v in cmpl_model_dict.items():
        n_comunidad = 1
        for comunidad in v.keys():
            print('Tienes', len(cmpl_model_dict), 'productos y vas por el', vuelta_general, 
                  '. Este producto tiene', len(v), 'comunidades y vas por la', n_comunidad,
                 '. La variable es', variable)
            timeseries_data = cmpl_model_dict[k][comunidad][[variable, 'date']].copy()
            timeseries_data.index = pd.DatetimeIndex(timeseries_data.date)
            timeseries_data.drop('date', axis = 1, inplace = True)
            timeseries_data.sort_index(inplace = True)

            train = timeseries_data.loc[timeseries_data.index <= '2020-02-01']
            test = timeseries_data.loc[timeseries_data.index > '2020-02-01']

            mod = SARIMAX(train, 
                order = [p, d, q],
                seasonal_order = [P, D, Q, m], 
                enforce_stationarity = False,
                enforce_invertibility = False)

            result = mod.fit()
            ypred = result.get_forecast(steps = len(test))
            ypred_df = pd.DataFrame(ypred.predicted_mean).rename(columns = {'predicted_mean': str(variable) + '_predicted'})
            pred_ci = ypred.conf_int()
            pred_ci_df = pd.concat([ypred_df, pred_ci], axis = 1)
            pred_ci_df[['ccaa', 'producto']] = comunidad, k

            predicciones = pd.concat([predicciones, pred_ci_df], axis = 0)
            n_comunidad += 1
            clear_output(wait = True)
        vuelta_general += 1
    return predicciones

In [7]:
df_peque = best_sarima('volumen_miles_de_kg')
df_medi = pd.concat([df_peque, best_sarima('valor_miles_de_€')], axis = 1)
df_gran = pd.concat([df_medi, best_sarima('precio_medio_kg')], axis = 1)

Tienes 50 productos y vas por el 50 . Este producto tiene 18 comunidades y vas por la 18 . La variable es precio_medio_kg


In [9]:
df_final = df_gran.T.drop_duplicates().T
df_final.to_csv(os.path.join(dir_out, 'predicciones_consumo.csv'), sep = ';')